In [14]:
import numpy as np
from scipy.spatial.distance import jensenshannon
import h5py
import json
import pandas as pd
import h5py
import pyBigWig
import matplotlib
from tqdm import tqdm
import scipy.ndimage
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
import math
import pandas as pd
import json 

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42 

In [15]:
def fetch_coorecetd_jsd_values(main_dir,cell_type,dtype,fold_info):
    
    #file1= main_dir+"/chrombpnet_model/chrombpnet_predictions.h5"
    #bigwig="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/"+dtype+"/"+cell_type+"/data/"+cell_type+"_unstranded.bw"
    #peaks=main_dir+"/chrombpnet_model/filtered.peaks.bed"
    #nonpeaks=main_dir+"/chrombpnet_model/filtered.nonpeaks.bed"
    #chr_fold_path="/mnt/lab_data2/anusri/chrombpnet/splits/"+fold_info+".json"
 
    file1= main_dir+"/chrombpnet_model/bias_predictions.h5"
    bigwig="/mnt/lab_data2/anusri/chrombpnet/results/chrombpnet/"+dtype+"/"+cell_type+"/data/"+cell_type+"_unstranded.bw"
    peaks=main_dir+"/chrombpnet_model/filtered.peaks.bed"
    nonpeaks=main_dir+"/chrombpnet_model/filtered.nonpeaks.bed"
    chr_fold_path="/mnt/lab_data2/anusri/chrombpnet/splits/"+fold_info+".json"
 
    
    def jsd_min_max_bounds(profile, mean_profile):
        """
            Min Max bounds for the jsd metric

            Args:
                profile (numpy.ndarray): true profile 

            Returns:
                tuple: (min, max) bounds values
        """

        # uniform distribution profile
        #uniform_profile = np.ones(len(profile)) * (1.0 / len(profile))
        uniform_profile = mean_profile

        # profile as probabilities
        profile_prob = profile / np.sum(profile)

        # jsd of profile with uniform profile
        max_jsd = jensenshannon(profile_prob, uniform_profile)

        # jsd of profile with itself (upper bound)
        min_jsd = 0.0

        return (min_jsd, max_jsd)


    def get_min_max_normalized_value(val, minimum, maximum):
        ret_val = (val - maximum) / (minimum - maximum)
        
        #print(ret_val, val, minimum, maximum)
        if ret_val <= 0:
            return 0
    
        if ret_val >= 1:
            return 1

        return ret_val
    
    data = h5py.File(file1)

    mode="test"
    splits_dict=json.load(open(chr_fold_path))
    chroms_to_keep=splits_dict[mode]

    NARROWPEAK_SCHEMA = ["chr", "start", "end", "1", "2", "3", "4", "5", "6", "summit"]
    peaks_bed_regions=pd.read_csv(peaks,header=None,sep='\t',names=NARROWPEAK_SCHEMA)
    peaks_bed_regions_to_keep=peaks_bed_regions[peaks_bed_regions["chr"].isin(chroms_to_keep)]

    NARROWPEAK_SCHEMA = ["chr", "start", "end", "1", "2", "3", "4", "5", "6", "summit"]
    nonpeaks_bed_regions=pd.read_csv(nonpeaks,header=None,sep='\t',names=NARROWPEAK_SCHEMA)
    nonpeaks_bed_regions_to_keep=nonpeaks_bed_regions[nonpeaks_bed_regions["chr"].isin(chroms_to_keep)]

    print(file1)
    print(data.keys())

    profile_predictions = data["predictions"]["profs"].value
    prof_coords_chr = data["coords"]["coords_chrom"].value
    prof_coords_center = data["coords"]["coords_center"].value
    prof_coords_peak = data["coords"]["coords_peak"].value


    bw = pyBigWig.open(bigwig) 

    true_bounts_mat_peaks = []
    true_bounds_mat_peaks_non_peaks = []
    jsd_only = []

    idx=0
    
    avg_true_counts_mat = []
    for jdx in tqdm(range(prof_coords_chr.shape[0])):
            if prof_coords_peak[jdx] == 0:
                    continue

            chr1 = peaks_bed_regions_to_keep.iloc[jdx, 0]
            start = peaks_bed_regions_to_keep.iloc[jdx, 1] + peaks_bed_regions_to_keep.iloc[jdx, -1] - 500
            end  =  peaks_bed_regions_to_keep.iloc[jdx, 1] + peaks_bed_regions_to_keep.iloc[jdx, -1] + 500

            true_counts = np.nan_to_num(bw.values(chr1,start,end ))
            avg_true_counts_mat.append(true_counts)
 
    mean_profile_label=np.nanmean(avg_true_counts_mat,axis=0)
    mean_profile_label_prob=mean_profile_label/np.nansum(mean_profile_label)

    for jdx in tqdm(range(prof_coords_chr.shape[0])):
            if prof_coords_peak[jdx] == 0:
                    chr1 = nonpeaks_bed_regions_to_keep.iloc[idx, 0]
                    start = nonpeaks_bed_regions_to_keep.iloc[idx, 1] + nonpeaks_bed_regions_to_keep.iloc[idx, -1] - 500
                    end  =  nonpeaks_bed_regions_to_keep.iloc[idx, 1] + nonpeaks_bed_regions_to_keep.iloc[idx, -1] + 500

                    true_counts = np.nan_to_num(bw.values(chr1,start,end ))
                    bounds = jsd_min_max_bounds(true_counts, mean_profile_label_prob)
                    pred_probs=profile_predictions[jdx]
                    pseudocount=0.001
                    cur_jsd=jensenshannon(true_counts/(pseudocount+np.nansum(true_counts)),pred_probs)
                    new_val = get_min_max_normalized_value(cur_jsd,bounds[0],bounds[1])
                    true_bounds_mat_peaks_non_peaks.append(new_val)
                    idx += 1
                    #jsd_only.append(cur_jsd)
                    continue

            chr1 = peaks_bed_regions_to_keep.iloc[jdx, 0]
            start = peaks_bed_regions_to_keep.iloc[jdx, 1] + peaks_bed_regions_to_keep.iloc[jdx, -1] - 500
            end  =  peaks_bed_regions_to_keep.iloc[jdx, 1] + peaks_bed_regions_to_keep.iloc[jdx, -1] + 500

            true_counts = np.nan_to_num(bw.values(chr1,start,end ))
            bounds = jsd_min_max_bounds(true_counts, mean_profile_label_prob)
            pred_probs=profile_predictions[jdx]
            pseudocount=0.001
            cur_jsd=jensenshannon(true_counts/(pseudocount+np.nansum(true_counts)),pred_probs)
            new_val = get_min_max_normalized_value(cur_jsd,bounds[0],bounds[1])
            true_bounts_mat_peaks.append(new_val)
            jsd_only.append(cur_jsd)

            
    #return np.nanmedian(true_bounts_mat_peaks), np.nanmedian(jsd_only), np.nanmean(true_bounts_mat_peaks), np.nanmean(jsd_only)
    return np.nanmedian(true_bounds_mat_peaks_non_peaks), np.nanmedian(jsd_only), np.nanmean(true_bounds_mat_peaks_non_peaks), np.nanmean(jsd_only)

    


In [16]:
meta = pd.read_csv("/mnt/lab_data2/anusri/chrombpnet/model_dir_atac.csv",header=None)

In [17]:
meta

,0,1,2
0,fold_0,GM12878,/mnt/lab_data2/anusri/chrombpnet/results/chrom...
1,fold_1,GM12878,/oak/stanford/groups/akundaje/projects/chromat...
2,fold_2,GM12878,/oak/stanford/groups/akundaje/projects/chromat...
3,fold_3,GM12878,/oak/stanford/groups/akundaje/projects/chromat...
4,fold_4,GM12878,/oak/stanford/groups/akundaje/projects/chromat...
5,fold_0,K562,/mnt/lab_data2/anusri/chrombpnet/results/chrom...
6,fold_1,K562,/oak/stanford/groups/akundaje/projects/chromat...
7,fold_2,K562,/oak/stanford/groups/akundaje/projects/chromat...
8,fold_3,K562,/oak/stanford/groups/akundaje/projects/chromat...
9,fold_4,K562,/oak/stanford/groups/akundaje/projects/chromat...


In [34]:
import subprocess
for i,r in meta.iterrows():
    main_dir=r[2]
    cell_type=r[1]
    dtype="DNASE_SE"
    if cell_type=="K562":
        dtype="DNASE_PE"
    if cell_type=="HEPG2":
        dtype="DNASE_PE"
    fold_info = r[0]
        
    #dtype="ATAC_PE"
    if cell_type=="GM12878":
        continue
  
    if cell_type=="K562":
        continue

    try:
        file1= main_dir+"/chrombpnet_model/bias_predictions.h5"
        data = h5py.File(file1, "r")
        profile_predictions = data["predictions"]["profs"].value
    except:
        if main_dir[-1] == "/":
            main_dir = main_dir[0:-1]
        folder_n = main_dir.strip().replace("//","/").strip().split("/")[-1]
        print(main_dir.strip().replace("//","/").split("/"))
        print(folder_n)
        command = 'kubectl cp populate-chrombpnet-pod:/chrombpnet/ATAC_PE/'+cell_type+'/'+folder_n+"/chrombpnet_model/bias_predictions.h5 "+file1
        print(command)
        p = subprocess.Popen([command], shell=True)
        p.wait()
        

    coorected_values = fetch_coorecetd_jsd_values(main_dir,cell_type,dtype,fold_info)
    print(coorected_values)
 
    #outf=main_dir+"/chrombpnet_model/chrombpnet_metrics_updated.json"
    outf=main_dir+"/chrombpnet_model/bias_metrics_updated.json"

    print(outf)
    #data = json.load(open(main_dir+"/chrombpnet_model/chrombpnet_metrics.json"))
    data = json.load(open(main_dir+"/chrombpnet_model/bias_metrics.json"))

    new_data = data.copy()
    new_data["profile_metrics"]["nonpeaks"]["mean_norm_jsd"] = coorected_values[2]
    new_data["profile_metrics"]["nonpeaks"]["median_norm_jsd"] = coorected_values[0]
    json.dump(new_data,open(outf,"w"),indent=4)
    

['', 'oak', 'stanford', 'groups', 'akundaje', 'projects', 'chromatin-atlas-2022', 'chrombpnet', 'folds', 'ATAC', 'HEPG2', 'HEPG2_05.09.2022_bias_128_4_1234_0.8_fold_0']
HEPG2_05.09.2022_bias_128_4_1234_0.8_fold_0
kubectl cp populate-chrombpnet-pod:/chrombpnet/ATAC_PE/HEPG2/HEPG2_05.09.2022_bias_128_4_1234_0.8_fold_0/chrombpnet_model/bias_predictions.h5 /oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/ATAC/HEPG2/HEPG2_05.09.2022_bias_128_4_1234_0.8_fold_0//chrombpnet_model/bias_predictions.h5


/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


/oak/stanford/groups/akundaje/projects/chromatin-atlas-2022/chrombpnet/folds/ATAC/HEPG2/HEPG2_05.09.2022_bias_128_4_1234_0.8_fold_0/chrombpnet_model/bias_predictions.h5
<KeysViewHDF5 []>


KeyError: "Unable to open object (object 'predictions' doesn't exist)"